In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import time
from datetime import datetime, timedelta, timezone
import re
import csv
import sqlite3
import os
import configparser


In [2]:
# Connect to the SQLite database (create a new one if it doesn't exist)
conn = sqlite3.connect("scraped_prices.db")
cursor = conn.cursor()

In [3]:
# # Execute DELETE statement to clear all rows from the table
# cursor.execute(f"DROP TABLE IF EXISTS {'receipts'}")

# # Commit the changes to the database
# cursor.connection.commit()

In [4]:
# Replace these with your actual credentials and website URL
config = configparser.ConfigParser()
config.read('config.ini')

username = config['Credentials']['USERNAME']
password = config['Credentials']['PASSWORD']

website_url = "https://www.costco.ca/LogonForm"

In [5]:

# Specify the path to chromedriver
# service = Service(r'C:\\Users\\Jacky Sio\\costco_price_scraper\\chromedriver.exe')

# Create ChromeOptions and specify the browser binary
# options = webdriver.ChromeOptions()
# options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
# options.add_argument("--headless")
# options.add_experimental_option("excludeSwitches", ["enable-automation"])

# Create the webdriver with the specified options
#driver = webdriver.Chrome(service=service, options=options)
driver = uc.Chrome(use_subprocess=False)
# Open the website
driver.get(website_url)
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@id="next"]')))
time.sleep(4)
# Find the username and password input fields and submit button
username_field = driver.find_element("id", "signInName")
password_field = driver.find_element("id", "password")
sign_in_button = driver.find_element(By.XPATH, '//button[@id="next"]')




In [6]:
# Enter your credentials and submit the form
username_field.send_keys(username)
username_field.clear()
time.sleep(1)
username_field.send_keys(username)
time.sleep(1.1)
password_field.send_keys(password)
password_field.clear()
time.sleep(2.4)
password_field.send_keys(password)
ActionChains(driver).move_to_element(sign_in_button).perform()  


In [7]:
time.sleep(5.3)
ActionChains(driver).double_click(sign_in_button).perform()
try:
    time.sleep(1)
    sign_in_button.click()
    time.sleep(5)
except Exception:
    print("Timeout: The next page did not load within the expected time.")


In [8]:
time.sleep(3)
try:
    # Wait for the presence of an element on the next page (change the selector accordingly)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "search-field"))
    )

    # At this point, you can perform additional checks or actions on the next page
    # For example, check for a welcome message or other elements to confirm successful sign-in

    print("Sign-in successful!")

except Exception:
    print("Timeout: The next page did not load within the expected time.")

Sign-in successful!


In [9]:
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,"header_order_and_returns")))
account_button = driver.find_element("id","header_order_and_returns")
#time.sleep(6)
account_button.click()
#time.sleep(5)

In [10]:
#WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[@id="next"]')))
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH,'//button[@automation-id="myWarehouseOrdersTab"]')))
warehouse_button = driver.find_element(By.XPATH,'//button[@automation-id="myWarehouseOrdersTab"]')
warehouse_button.click()
time.sleep(3)


### Database Code

In [11]:
# Create the Receipts table
def create_receipts_table(cursor):
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS receipts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        item_id INT,
        item_name TEXT,
        item_price REAL,
        date DATE,
        receipt_id TEXT,
        on_sale BOOLEAN,
        receipt_path TEXT
        )
''')

In [12]:

def get_all_receipt_ids(cursor):
    cursor.execute("SELECT DISTINCT receipt_id FROM receipts")
    result = cursor.fetchall()
    receipt_ids = [row[0] for row in result]
    return receipt_ids

In [13]:
def get_all_item_ids_not_on_sale(cursor):
    cursor.execute("SELECT DISTINCT item_id FROM receipts WHERE on_sale = 0")
    result = cursor.fetchall()
    item_ids = [row[0] for row in result]
    return item_ids

In [14]:
def upsert_receipt_data(cursor, all_receipt_items_list):

    for receipt_item in all_receipt_items_list:
        item_id = receipt_item.get("item_id")
        item_name = receipt_item.get("item_name")
        item_price = receipt_item.get("item_price")
        date = receipt_item.get("date")
        receipt_id = receipt_item.get("receipt_id")
        on_sale = receipt_item.get("on_sale")
        receipt_path = receipt_item.get("receipt_path")
        # Upsert data using INSERT OR REPLACE
        cursor.execute('''
            INSERT OR REPLACE INTO receipts (item_id, item_name, item_price, date, receipt_id, on_sale, receipt_path)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (item_id, item_name, item_price, date, receipt_id, on_sale, receipt_path))

In [15]:


create_receipts_table(cursor)



In [16]:
def process_receipt_metadata(all_receipt_ids_set):
    
     # Locate the table element using its HTML structure
    receipt_element = driver.find_element(By.ID,'dataToPrint')
    # Get the HTML content of the table
    receipt_html = receipt_element.get_attribute('outerHTML')
    soup = BeautifulSoup(receipt_html, 'html.parser')
    
    receipt_id = soup.find_all('div', class_="MuiBox-root css-11s8ayx")
    if receipt_id:
        receipt_id = receipt_id[-1].get_text(strip=True)
        print(receipt_id)
        # Check if receipt_id is in the set of all_receipt_ids_set
        if receipt_id in all_receipt_ids_set:
            print(f"Receipt {receipt_id} is already processed. Skipping...")
            return None, None, None  # Skip processing further
    date_str = soup.find('span', class_="date MuiBox-root css-ke5oan")
    time_str = soup.find('span', class_="time MuiBox-root css-5c53yh")
    if date_str is not None and time_str is not None:
        date_str = date_str.get_text(strip=True)
        time_str = time_str.get_text(strip=True)
        print(date_str)
    date_time_str = f"{date_str} {time_str}"
    date_time_format = "%m/%d/%Y %H:%M"
    date_time_obj = datetime.strptime(date_time_str, date_time_format)
    print(date_time_obj)
    
    receipts_folder = "receipts"
    if not os.path.exists(receipts_folder):
        os.makedirs(receipts_folder)
    
    # Generate a safe file name by replacing invalid characters
    filename_safe_str = re.sub(r'[^a-zA-Z0-9]', '_', date_time_str)
    file_name = f"receipt_{filename_safe_str}_{receipt_id}.png"
    receipt_path = os.path.join(receipts_folder, file_name)
    # Save the screenshot with the safe file name
    driver.save_screenshot(receipt_path)
    print("Screenshot saved as:", receipt_path)
    
    return receipt_id, date_time_str, receipt_path

In [17]:
def check_for_discount_prefix(name):
    match = re.match(r"TPD/(\d+)", name)
    if match:
        return match.group(1)
    else:
        return None

In [18]:
def process_receipt_items(receipt_id, date_time_str, receipt_path):
        # Locate the table element using its HTML structure
    table_element = driver.find_element(By.CSS_SELECTOR,'table.MuiTable-root[aria-label="spanning table"]')
    # Get the HTML content of the table
    table_html = table_element.get_attribute('outerHTML')
    soup = BeautifulSoup(table_html, 'html.parser')
    item_list = []
    discount_set = set()
    table_body = soup.find('tbody', class_='MuiTableBody-root')
    for row in table_body.find_all('tr', class_='MuiTableRow-root css-ufft4h'):
        # Extract data from each column in the row
        item_id = row.find('td', class_='css-tedx13')
        item_name = row.find('td', class_='css-u9y9s5')
        item_price = row.find('td', class_='css-1879r0q')
        # Extract ID, name, and price
        if item_id is not None and item_name is not None and item_price is not None:
            id = item_id.get_text(strip=True)
            name = item_name.get_text(strip=True)
            #check for discounts
            discount_id = check_for_discount_prefix(name)
            if discount_id is not None:
                discount_set.add(discount_id)
                continue
            price = item_price.get_text(strip=True)
            item_info = {'item_id': id, 'item_name': name, 'item_price': price, 'receipt_id': receipt_id, 'date': date_time_str, 'on_sale': False, 'receipt_path':receipt_path}
            item_list.append(item_info)
            # Perform desired actions, e.g., print or store in a list
            print(f"ID: {id}, Name: {name}, Price: {price}")
        else:
            break
    for item in item_list:
        item_id = item['item_id']
        if item_id in discount_set:
            item['on_sale'] = True
    
    return item_list

In [19]:
view_receipt_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[automation-id="ViewInWareHouseReciept"]')
all_receipt_items_list = []
for index, button in enumerate(view_receipt_buttons, start=1):
    print(f"Clicking 'View Receipt' button {index}")
    button.click()
    time.sleep(2)
    all_receipt_ids_set = set(get_all_receipt_ids(cursor))
    receipt_id, date_time_str, receipt_path = process_receipt_metadata(all_receipt_ids_set)
    if receipt_id is not None:
        all_receipt_items_list+=process_receipt_items(receipt_id, date_time_str, receipt_path)
    pop = driver.find_element(By.CSS_SELECTOR, 'button.MuiButtonBase-root[aria-label="Close"]')
    pop.click()
    scroll_script = "window.scrollBy(0, 200);"
    driver.execute_script(scroll_script)

Clicking 'View Receipt' button 1
22054800601682312281221
Receipt 22054800601682312281221 is already processed. Skipping...
Clicking 'View Receipt' button 2
22054812300662312281157
Receipt 22054812300662312281157 is already processed. Skipping...
Clicking 'View Receipt' button 3
22054800302752312231317
Receipt 22054800302752312231317 is already processed. Skipping...
Clicking 'View Receipt' button 4
22054800700622312191005
Receipt 22054800700622312191005 is already processed. Skipping...
Clicking 'View Receipt' button 5
22054801106272312182051
Receipt 22054801106272312182051 is already processed. Skipping...
Clicking 'View Receipt' button 6
22054819101342312182025
Receipt 22054819101342312182025 is already processed. Skipping...
Clicking 'View Receipt' button 7
22054801203912312131504
Receipt 22054801203912312131504 is already processed. Skipping...
Clicking 'View Receipt' button 8
22054812101402312131412
Receipt 22054812101402312131412 is already processed. Skipping...
Clicking 'View R

In [20]:
conn.commit()

In [21]:
print(all_receipt_items_list)
upsert_receipt_data(cursor, all_receipt_items_list)
all_items_list = get_all_item_ids_not_on_sale(cursor)

[]


In [22]:
print(all_items_list)

[743244, 1779962, 1794497, 30669, 13175, 1533852, 30739, 1570675, 4223366, 29464, 1743411, 1780268, 11521, 8068, 5675, 399299, 6485, 9481, 1588626, 9484, 170547, 1779820, 2280539, 1601370, 31408, 1647794, 1579298, 67245, 9134, 290291, 251813, 532683, 692731, 1641849, 252214, 23106, 879639, 31098, 15764, 262282, 2142667, 1601313, 2551586, 5051501, 941712, 347937, 6283006, 401150, 6283010, 5588009, 675749, 1794316, 6283008, 969786, 400615, 348, 1564000, 203435, 1652577, 13813, 804449, 1037794, 1856100, 1801236, 2531001, 1792087, 4531001, 1792088, 2197001, 695035, 1303489, 1433048, 2619, 2531000, 313963, 1601331, 1797778, 8512544, 1149838, 234745, 1101646, 1480774, 242846]


In [23]:
import requests

# URL of your Flask API
api_url = "http://localhost:5000/check_sale"  # Update with your actual URL

# Example item IDs


# JSON payload
payload = {"item_ids": all_items_list}

# Make a POST request
response = requests.post(api_url, json=payload)

# Check the response
if response.status_code == 200:
    data = response.json()
    print("Total Savings:", data["total_savings"])
    print("Sale Info:")
    for sale_item in data["sale_info"]:
        print(f"Item ID: {sale_item['item_id']}")
        print(f"Item Name: {sale_item['item_name']}")
        print(f"Savings: {sale_item['savings']}")
        print(f"Expiry Date: {sale_item['expiry_date']}")
        print(f"Sale Price: {sale_item['sale_price']}")
        print("---")
else:
    print("Request failed with status code:", response.status_code)
    print(response.text)

Total Savings: 0
Sale Info:


In [24]:
# view_receipt_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[automation-id="ViewInWareHouseReciept"]')

# for index, button in enumerate(view_receipt_buttons, start=1):
#     #view_receipt = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[automation-id="ViewInWareHouseReciept"]')))
#     print(f"Clicking 'View Receipt' button {index}")
#     button.click()
#     time.sleep(1)
#     driver.save_screenshot(f"receipt{index}.png")
#     pop = driver.find_element(By.CSS_SELECTOR, 'button.MuiButtonBase-root[aria-label="Close"]')
#     pop.click()
#     scroll_script = "window.scrollBy(0, 200);"
#     driver.execute_script(scroll_script)

In [25]:
# Commit the changes and close the connection
conn.commit()
conn.close()